### Домашнее задание к лекции 1 Введение в Feature engineering

**Задание:**

Необходимо запустить практическую часть занятия- файл находиться в материалых, и посмотреть самому то, о чём говорили на лекции.
По образу практики, попробуйте создать искусственный датасет с лишними столбцами.
Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель.
Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown.
Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!


**Решение:**

Создадим искусственный датасет, который описывает продажи в магазине, в зависимости от следующих факторов:
- затраты на рекламу;
- затраты на персонал;
- стоимость аренды;
- ставка аренды за кв м, чем выше - тем более качественная локация, приносящая больше траффика;
- площадь магазина;

В данном случае стоимость аренды с одной стороны и ставка аренды + площадь с другой - зависимы, то есть, мы предполагаем, что без ущерба для модели можно брать либо первое, либо пару вторых. 

Затраты на персонал могут оказывать влияние, а могут и не влиять в реальности, но в нашей условной модели считаем - не влияют


In [0]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import statistics

In [0]:
n_samp = 500
atl = np.random.choice(900, n_samp)
staff = np.random.choice(200, n_samp) + 50
rent = np.random.choice(2000, n_samp) + 100
rent_price = np.random.choice(200, n_samp) + 100
area = np.random.choice(10, n_samp) + 10

turnover = atl * rent * 50



In [35]:
df = pd.DataFrame({'atl':atl, 'staff':staff, 'rent':rent, 'rent_price':rent_price, 'area':area, 'turnover':turnover})
df.head()

,atl,staff,rent,rent_price,area,turnover
0,876,177,666,132,19,29170800
1,63,163,1156,209,18,3641400
2,170,149,531,294,11,4513500
3,56,147,1021,267,15,2858800
4,29,65,1749,190,18,2536050


In [36]:
df.columns

Index(['atl', 'staff', 'rent', 'rent_price', 'area', 'turnover'], dtype='object')

In [37]:
y = df['turnover']
X1 = df[['atl', 'staff', 'rent', 'rent_price', 'area']]

reg = LinearRegression().fit(X1, y)

print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

Weights: [53524.20808424 -1803.73172992 21274.56248646  6110.33538013
 24889.91281052]
Bias: -24020256.929531798


In [38]:
predicted = reg.predict(X1)

print(f'Prediction error: {mean_absolute_error(predicted, y)}')
print(f'Relative comparison of the error and mean of turnover is: {round((100 * mean_absolute_error(predicted, y) / statistics.mean(y)), 2)}%')

Prediction error: 5480633.085334305
Relative comparison of the error and mean of turnover is: 22.2%


Модель линейной регрессии вывела уравнение зависимости вида (WeightsN - коэффициент из модели, N - порядковый номер в выводе ячейки ноутбука):
Weights1 * atl + Weights2 * staff + Weights3 * rent + Weights4 * rent_price + Weights5 * area - Bias = turnover +- Prediction error

Возьмем в качестве метрики следующую величину - Отношение вычисленной ошибки предсказания модели к среднему значению turnover, оно достаточно велико: 22.2%

Сравним точность предсказания модели, если убрать переменную staff

In [39]:
y = df['turnover']
X2 = df[['atl', 'rent', 'rent_price', 'area']]

reg = LinearRegression().fit(X2, y)
predicted = reg.predict(X2)

print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
print(f'Prediction error: {mean_absolute_error(predicted, y)}')
print(f'Relative comparison of the error and mean of turnover is: {round((100 * mean_absolute_error(predicted, y) / statistics.mean(y)), 2)}%')

Weights: [53523.85324086 21270.39445411  6017.58236593 26594.90717171]
Bias: -24291772.801218018
Prediction error: 5486749.432289083
Relative comparison of the error and mean of turnover is: 22.22%


Считаем, что изменения точности предсказания нет, сотые процента в данных условиях не могут считаться значимым изменением.

Следующий шаг - уберем еще переменные rent_price и area

In [40]:
y = df['turnover']
X3 = df[['atl', 'rent']]

reg = LinearRegression().fit(X3, y)
predicted = reg.predict(X3)

print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
print(f'Prediction error: {mean_absolute_error(predicted, y)}')
print(f'Relative comparison of the error and mean of turnover is: {round((100 * mean_absolute_error(predicted, y) / statistics.mean(y)), 2)}%')

Weights: [53559.33814522 21326.62141989]
Bias: -22768398.931502074
Prediction error: 5481606.639222896
Relative comparison of the error and mean of turnover is: 22.2%


Дальнейшее уменьшение переменных практически не влияет на точность предсказания модели, изменения в пределах сотых выбранной нами метрики.

Почему предсказание модели по прежнему не точное? Поскольку мы точно знаем, как задавали вычисление turnover в нашем датафрейме, то мы также знаем, что зависимость целевого показателя от переменных не является линейным уравнением. 

Сформулируем гипотезу: Если, задать при формировании датасета четкую линейную зависимость переменных и целевого показателя, модель линейной регрессии найдет формулу с высокой точностью. 

Проверим гипотезу:

In [45]:
turnover2 = atl * 20 + rent * 50 + 1000
df = pd.DataFrame({'atl':atl, 'staff':staff, 'rent':rent, 'rent_price':rent_price, 'area':area, 'turnover2':turnover2})
df.head()

,atl,staff,rent,rent_price,area,turnover2
0,876,177,666,132,19,51820
1,63,163,1156,209,18,60060
2,170,149,531,294,11,30950
3,56,147,1021,267,15,53170
4,29,65,1749,190,18,89030


In [46]:
y = df['turnover2']
X = df[['atl', 'rent', 'rent_price', 'area']]

reg = LinearRegression().fit(X, y)
predicted = reg.predict(X)

print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
print(f'Prediction error: {mean_absolute_error(predicted, y)}')
print(f'Relative comparison of the error and mean of turnover is: {round((100 * mean_absolute_error(predicted, y) / statistics.mean(y)), 2)}%')

Weights: [ 2.00000000e+01  5.00000000e+01 -1.77635684e-15  1.31228362e-13]
Bias: 999.9999999999491
Prediction error: 1.9761500880122186e-11
Relative comparison of the error and mean of turnover is: 0.0%


Как видим, если ввести четкую линейную зависимость, модель регрессии найдет коэффициенты уравнения и в том случае, если есть лишние переменные. Коэффициенты при лишних переменных очень малы, десять в -15 и -13 степени, а коэффициенты при значащих переменных и свободный член уравнения найдены с идеальной точностью.

Конечно, в общем случае, мы не можем считать, что проверка гипотезы на одном датасете - это достаточное доказательство. В условиях реального эксперимента необходимо было бы проверить достаточное количество датасетов с получением устойчивого результата. В условиях нашего эксперимента факт того, что модель линейной регрессии хорошо работает с данными, имеющими линейную зависимость, очевиден. 